In [26]:
import ssl
import certifi

ssl._create_default_https_context = ssl._create_unverified_context  # ข้ามการตรวจสอบ (ถ้าจำเป็น)

# หรือกำหนดค่าให้ใช้ CA Certificate ที่ถูกต้อง
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())


In [ ]:
import re
import os
import string
import pandas as pd
import time
import numpy as np
import fasttext.util
import emoji
import nltk
import tensorflow as tf
import mlflow
import mlflow.tensorflow

from huggingface_hub import hf_hub_download
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import KFold
from num2words import num2words
from unidecode import unidecode
from datasets import load_dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import TFAutoModel

from tqdm import tqdm
from argparse import Namespace


In [ ]:
# tensorboard --logdir logs
# http://localhost:6006

%load_ext tensorboard
%tensorboard --logdir logs

# Set log directory
log_dir = os.path.join("logs", "fit")

# TensorBoard Callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [64]:
args = Namespace(
    MODEL_STORAGE="storage_models",
    DATASET_DIR="dataset",
    INPUT_DIMENSION=50000,
    INPUT_LENGTH=300,
    OUTPUT_DIMENSION=300
)

In [44]:
# nltk.download("stopwords")
# nltk.download("punkt")
# nltk.download("wordnet")

nltk.data.path.append(f"{args.DATASET_DIR}/nltk_data")

In [ ]:
questions = squad_dataset["train"]["question"]
answers = squad_dataset["train"]["answers"]

# Ensure every answer has at least one text value
filtered_data = [(q, a["text"][0]) for q, a in zip(questions, answers) if len(a["text"]) > 0]

# Unzip filtered data
questions, answers = zip(*filtered_data)

# Compute sentence vectors efficiently
X = np.array([get_sentence_vector(clean_text(q)) for q in tqdm(questions, desc="Processing Questions")])
y = np.array([get_sentence_vector(clean_text(a)) for a in tqdm(answers, desc="Processing Answers")])

Processing Answers: 100%|██████████| 86821/86821 [04:43<00:00, 306.20it/s]


In [46]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

mlflow.set_experiment("QA_Comparison")

<Experiment: artifact_location='file:///Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/tensorflow_lab/mlruns/308254716828157495', creation_time=1740980879744, experiment_id='308254716828157495', last_update_time=1740980879744, lifecycle_stage='active', name='QA_Comparison', tags={}>

In [34]:
ft = fasttext.load_model(f'{args.MODEL_STORAGE}/fasttext-en-vector.300.bin')

In [37]:
squad_dataset = load_dataset("rajpurkar/squad_v2", trust_remote_code=True)
# ds = load_dataset("rajpurkar/squad_v2")

Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 2063094.35 examples/s]


In [39]:
def get_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    return sum(word_vectors) / len(word_vectors) if word_vectors else np.zeros(300)


In [47]:
CONTRACTIONS = {
    "can't": "cannot",
    "won't": "will not",
    "n't": " not",
    "'re": " are",
    "'s": " is",
    "'d": " would",
    "'ll": " will",
    "'t": " not",
    "'ve": " have",
    "'m": " am"
}

def expand_contractions(text, contractions_dict=CONTRACTIONS):
    """แปลงคำย่อให้เป็นคำเต็ม"""
    for contraction, full_form in contractions_dict.items():
        text = re.sub(contraction, full_form, text)
    return text

def replace_numbers(text):
    """แปลงตัวเลขเป็นคำ"""
    return re.sub(r'\b\d+\b', lambda x: num2words(int(x.group())), text)

def remove_special_characters(text):
    """ลบอักขระพิเศษ ยกเว้นตัวที่มีประโยชน์"""
    return re.sub(r'[^a-zA-Z0-9\s-]', '', text)  # คง "-" ไว้ในบางกรณี

def convert_emojis(text):
    """แปลงอิโมจิเป็นคำ"""
    return emoji.demojize(text, delimiters=(" ", " "))  # 😍 → smiling_face_with_heart-eyes

def clean_text(text):
    """ฟังก์ชันทำความสะอาดข้อความภาษาอังกฤษ พร้อมรองรับ Numbers, Emojis และ Special Characters"""
    text = text.lower()  # แปลงเป็นตัวพิมพ์เล็ก
    text = expand_contractions(text)  # ขยายคำย่อ
    text = convert_emojis(text)  # แปลงอิโมจิเป็นคำ
    text = unidecode(text)  # แปลงอักขระพิเศษ เช่น á → a
    text = replace_numbers(text)  # แปลงตัวเลขเป็นคำ
    text = remove_special_characters(text)  # ลบอักขระพิเศษที่ไม่ต้องการ
    words = word_tokenize(text)  # ตัดคำ (Tokenization)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization และลบ Stopwords
    text = " ".join(words).strip()  # รวมคำกลับเป็นข้อความ
    return text

sample_text = "I can't believe it's already 2024! NLP's growth is amazing. 😊 #AI @tech"
cleaned_text = clean_text(sample_text)
print(cleaned_text)


believe already two thousand twenty-four nlp growth amazing smilingfacewithsmilingeyes ai tech


In [ ]:
len(squad_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [50]:
questions = squad_dataset["train"]["question"]
answers = squad_dataset["train"]["answers"]

# Ensure every answer has at least one text value
filtered_data = [(q, a["text"][0]) for q, a in zip(questions, answers) if len(a["text"]) > 0]

# Unzip filtered data
questions, answers = zip(*filtered_data)

# Compute sentence vectors efficiently
X = np.array([get_sentence_vector(clean_text(q)) for q in tqdm(questions, desc="Processing Questions")])
y = np.array([get_sentence_vector(clean_text(a)) for a in tqdm(answers, desc="Processing Answers")])

Processing Answers: 100%|██████████| 86821/86821 [04:43<00:00, 306.20it/s]


In [54]:
# Ensure questions, answers, X, y exist
if 'questions' in locals() and 'answers' in locals() and 'X' in locals() and 'y' in locals():
    # Convert to DataFrame
    cleaned_df = pd.DataFrame({
        "question": questions,  # List of cleaned questions
        "answer": answers,      # List of cleaned answers
        "question_vector": list(X),  # Convert NumPy array to list
        "answer_vector": list(y)     # Convert NumPy array to list
    })

    # Save as Parquet file
    parquet_path = f"{args.DATASET_DIR}/cleaned_squad.parquet"
    cleaned_df.to_parquet(parquet_path, index=False, engine="pyarrow")

    print(f"✅ Cleaned dataset saved at: {parquet_path}")
else:
    print("❌ Error: Ensure `questions`, `answers`, `X`, and `y` are defined before saving.")


✅ Cleaned dataset saved at: dataset/cleaned_squad.parquet


In [55]:
cleaned_df

,question,answer,question_vector,answer_vector
0,When did Beyonce start becoming popular?,in the late 1990s,"[-0.012084373272955418, -0.00119769386947155, ...","[-0.02115001529455185, 0.022775297984480858, -..."
1,What areas did Beyonce compete in when she was...,singing and dancing,"[-0.014981376007199287, -0.01384357176721096, ...","[0.008058244362473488, 0.027559427544474602, 0..."
2,When did Beyonce leave Destiny's Child and bec...,2003,"[-0.0020953065250068903, 0.002123526530340314,...","[-0.010792444460093975, -0.006511264946311712,..."
3,In what city and state did Beyonce grow up?,"Houston, Texas","[-0.04700393229722977, -0.025392722338438034, ...","[0.009803368709981441, -0.0055719055235385895,..."
4,In which decade did Beyonce become famous?,late 1990s,"[-0.021190045401453972, 0.0027403663843870163,...","[-0.02115001529455185, 0.022775297984480858, -..."
...,...,...,...,...
86816,In what US state did Kathmandu first establish...,Oregon,"[0.018351297825574875, -0.051341600716114044, ...","[0.025934094563126564, -0.02484295703470707, 0..."
86817,What was Yangon previously known as?,Rangoon,"[-0.01408060360699892, 0.03888470679521561, -0...","[-0.05151166766881943, 0.003534462070092559, -..."
86818,With what Belorussian city does Kathmandu have...,Minsk,"[-0.024799088016152382, -0.009273444302380085,...","[0.004432872869074345, -0.04474014788866043, -..."
86819,In what year did Kathmandu create its initial ...,1975,"[-0.01714322157204151, -0.004151790868490934, ...","[-0.004570195451378822, -0.008268006145954132,..."


In [56]:
# K-Fold Cross Validation (5-fold)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

In [57]:
model_rnn = Sequential([
    Embedding(input_dim=50000, output_dim=300, input_length=300),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(128)),
    Dense(300, activation="relu")  # Output ขนาดเดียวกับ FastText
])

model_rnn.compile(optimizer="adam", loss="mse", metrics=["mae"])

/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/tensorflow_lab/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [58]:
model_cnn = Sequential([
    Embedding(input_dim=50000, output_dim=300, input_length=300),
    Conv1D(128, kernel_size=5, activation="relu"),
    GlobalMaxPooling1D(),
    Dense(300, activation="relu")
])

model_cnn.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [61]:
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [62]:
with mlflow.start_run():
    for epoch in tqdm(range(5), desc="Training Progress"):
        history = model_rnn.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val))
        
        mlflow.log_metric("train_loss", history.history["loss"][-1], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][-1], step=epoch)
    
    mlflow.tensorflow.log_model(model_rnn, "QA_BiLSTM")

Python(16541) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(16543) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(16545) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(16547) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

 471/2171 ━━━━━━━━━━━━━━━━━━━━ 16:53 596ms/step - loss: 0.0036 - mae: 0.0390

Training Progress:   0%|          | 0/5 [04:44<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, X_train, y_train, X_val, y_val):
    start_time = time.time()
    model.fit(X_train, y_train, epochs=3, validation_data=(X_val, y_val), verbose=0)
    train_time = time.time() - start_time
    loss, mae = model.evaluate(X_val, y_val, verbose=0)
    return mae, train_time

# รันเทียบ 3 โมเดล
mae_rnn, time_rnn = evaluate_model(model_rnn, X_train, y_train, X_val, y_val)
mae_cnn, time_cnn = evaluate_model(model_cnn, X_train, y_train, X_val, y_val)

print(f"RNN - MAE: {mae_rnn}, Training Time: {time_rnn:.2f}s")
print(f"CNN - MAE: {mae_cnn}, Training Time: {time_cnn:.2f}s")

In [ ]:
# Load best RNN model
best_rnn = load_model("checkpoints/best_rnn.h5")
best_cnn = load_model("checkpoints/best_cnn.h5")
best_transformer = load_model("checkpoints/best_transformer.h5")

# Verify models are loaded
print(best_rnn.summary())
print(best_cnn.summary())
print(best_transformer.summary())

In [ ]:
# Define checkpoint directory
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Early Stopping (patience = 3 epochs)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Checkpoints for each model
checkpoint_rnn = ModelCheckpoint(filepath=os.path.join(checkpoint_dir, "best_rnn.h5"), 
                                 save_best_only=True, monitor="val_loss", mode="min")

checkpoint_cnn = ModelCheckpoint(filepath=os.path.join(checkpoint_dir, "best_cnn.h5"), 
                                 save_best_only=True, monitor="val_loss", mode="min")

checkpoint_transformer = ModelCheckpoint(filepath=os.path.join(checkpoint_dir, "best_transformer.h5"), 
                                         save_best_only=True, monitor="val_loss", mode="min")


In [ ]:
with mlflow.start_run():
    for epoch in tqdm(range(5), desc="Training Progress"):
        # Train RNN Model
        history_rnn = model_rnn.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val),
                                    callbacks=[early_stopping, checkpoint_rnn])

        mlflow.log_metric("train_loss_rnn", history_rnn.history["loss"][-1], step=epoch)
        mlflow.log_metric("val_loss_rnn", history_rnn.history["val_loss"][-1], step=epoch)

        # Train CNN Model
        history_cnn = model_cnn.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val),
                                    callbacks=[early_stopping, checkpoint_cnn])

        mlflow.log_metric("train_loss_cnn", history_cnn.history["loss"][-1], step=epoch)
        mlflow.log_metric("val_loss_cnn", history_cnn.history["val_loss"][-1], step=epoch)

        # Train Transformer Model
        history_transformer = model_transformer.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val),
                                                    callbacks=[early_stopping, checkpoint_transformer])

        mlflow.log_metric("train_loss_transformer", history_transformer.history["loss"][-1], step=epoch)
        mlflow.log_metric("val_loss_transformer", history_transformer.history["val_loss"][-1], step=epoch)

    # Log the best models to MLflow
    mlflow.tensorflow.log_model(model_rnn, "QA_BiLSTM")
    mlflow.tensorflow.log_model(model_cnn, "QA_CNN")
    mlflow.tensorflow.log_model(model_transformer, "QA_Transformer")
